In [39]:
import librosa 
import numpy as np
import pandas as pd
import random
from augmented_dataset_generator import full_feature_with_norm
import yaml
import soundfile as sf
from sklearn import preprocessing
import os
from tqdm import tqdm

### General functions to create features, and choose which frames to replace

In [40]:
def local_scaling(x):
    """Scaling an array to fit between -1 and 1"""
    x_min = np.min(x)
    x_max = np.max(x)
    x_normed = (x - x_min) / (x_max - x_min)
    x_scaled = 2 * x_normed - 1
    
    return x_scaled

def extract_features(audio_data,
                     cfg = None,
                     data_config: str = './configs/salsa_lite_demo_3class.yml'
                    ) -> None:
    """
    Extract SALSA-Lite features from a segment of audio 
    SALSA-Lite consists of:
        - 4 Log-linear spectrograms 
        - 3 Normalized Interchannel Phase Differences 

    [This needs to be revised for the demo]
    The frequency range of log-linear spectrogram is 0 to 9kHz.

    Arguments
    -----------
    audio_data (np.ndarray) : audio data from mic streaming (assuming it is 4 channels)
    cfg (array)             : array of configuration parameters (from .yml file)
    data_config (.yml file) : filepath to the .yml config file used for SALSA-Lite

    Returns
    --------
    None 

    To-do
    ------
    """
    if cfg is None:
        # Load data config files
        with open(data_config, 'r') as stream:
            try:
                cfg = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
    # Parse config file
    fs = cfg['data']['fs']
    n_fft = cfg['data']['n_fft']
    hop_length = cfg['data']['hop_len']
    win_length = cfg['data']['win_len']

    # Doa info
    n_mics = 4
    fmin_doa = cfg['data']['fmin_doa']
    fmax_doa = cfg['data']['fmax_doa']
    
    """
    For the demo, fmax_doa = 4kHz, fs = 48kHz, n_fft = 512, hop = 300
    This results in the following:
        n_bins      = 257
        lower_bin   = 1
        upper_bin   = 42
        cutoff_bin  = 96 
        logspecs -> 95 bins total
        phasespecs -> 41 bins total
        
    Since these are all fixed, can we just put them into the config.yml instead
    and just read them from there and avoid these calculations
    """
    fmax_doa = np.min((fmax_doa, fs // 2))
    n_bins = n_fft // 2 + 1
    lower_bin = int(np.floor(fmin_doa * n_fft / float(fs)))
    upper_bin = int(np.floor(fmax_doa * n_fft / float(fs)))
    lower_bin = np.max((1, lower_bin))

    # Cutoff frequency for spectrograms
    fmax = 9000  # Hz, meant to reduce feature dimensions
    cutoff_bin = int(np.floor(fmax * n_fft / float(fs)))  # 9000 Hz, 512 nfft: cutoff_bin = 192
    assert upper_bin <= cutoff_bin, 'Upper bin for spatial feature is higher than cutoff bin for spectrogram!'

    # Normalization factor for salsa_lite --> 2*pi*f/c
    c = 343
    delta = 2 * np.pi * fs / (n_fft * c)
    freq_vector = np.arange(n_bins)
    freq_vector[0] = 1
    freq_vector = freq_vector[:, None, None]  # n_bins x 1 x 1
    
    # Extract the features from the audio data
    """
    The audio data is from the ambimik, assuming that it is a four-channel array
    The shape should be (4 , x) for (n_channels, time*fs)
    """
    log_specs = []
    for imic in np.arange(n_mics):
        # audio_mic_data = local_scaling(audio_data[imic, :]) 
        audio_mic_data = audio_data[imic, :]
        stft = librosa.stft(y=np.asfortranarray(audio_mic_data), 
                            n_fft=n_fft, 
                            hop_length=hop_length,
                            center=True, 
                            window='hann', 
                            pad_mode='reflect')
        if imic == 0:
            n_frames = stft.shape[1]
            X = np.zeros((n_bins, n_frames, n_mics), dtype='complex')  # (n_bins, n_frames, n_mics)
        X[:, :, imic] = stft
        # Compute log linear power spectrum
        spec = (np.abs(stft) ** 2).T
        log_spec = librosa.power_to_db(spec, ref=1.0, amin=1e-10, top_db=None)
        log_spec = np.expand_dims(log_spec, axis=0)
        log_specs.append(log_spec)
    log_specs = np.concatenate(log_specs, axis=0)  # (n_mics, n_frames, n_bins)

    # Compute spatial feature
    # X : (n_bins, n_frames, n_mics) , NIPD formula : -(c / (2pi x f)) x arg[X1*(t,f) . X2:M(t,f)]
    phase_vector = np.angle(X[:, :, 1:] * np.conj(X[:, :, 0, None]))
    phase_vector = phase_vector / (delta * freq_vector)
    phase_vector = np.transpose(phase_vector, (2, 1, 0))  # (n_mics, n_frames, n_bins)
    
    # Crop frequency
    log_specs = log_specs[:, :, lower_bin:cutoff_bin]
    phase_vector = phase_vector[:, :, lower_bin:cutoff_bin]
    phase_vector[:, :, upper_bin:] = 0
    
    # Stack features
    audio_feature = np.concatenate((log_specs, phase_vector), axis=0)
    audio_feature = audio_feature.astype(np.float32)
    
    # Now we normalize the first 4 log power spectrogram channels of SALSALITE
    n_feature_channels = 4
    scaler_dict = {}
    for i_chan in np.arange(n_feature_channels):
        scaler_dict[i_chan] = preprocessing.StandardScaler()
        scaler_dict[i_chan].partial_fit(audio_feature[i_chan, : , : ]) # (n_timesteps, n_features)
        
    # Extract mean and std
    feature_mean = []
    feature_std = []
    for i_chan in range(n_feature_channels):
        feature_mean.append(scaler_dict[i_chan].mean_)
        feature_std.append(np.sqrt(scaler_dict[i_chan].var_))

    feature_mean = np.array(feature_mean)
    feature_std = np.array(feature_std)

    # Expand dims for timesteps: (n_chanels, n_timesteps, n_features)
    feature_mean = np.expand_dims(feature_mean, axis=1)
    feature_std = np.expand_dims(feature_std, axis=1)
    audio_feature[:4] = (audio_feature[:4] - feature_mean)/feature_std
        
    return audio_feature

In [25]:
def get_random_index():
    return random.randint(0,4900)

def get_random_numframes_replace():
    return random.randint(0,7)

def get_mask(i):
    if i == 0:
        return [True, False, False, False, False]
    elif i == 1:
        return [False, True, True, True, True]
    elif i == 2:
        return [False, False, True, True, True]
    elif i == 3:
        return [False, False, False, True, True]
    elif i == 4:
        return [False, False, False, False, True]
    elif i == 5:
        return [True, True, True, True, False]
    elif i == 6:
        return [True, True, True, False, False]
    elif i == 7:
        return [True, True, False, False, False]

### Concat all noise data together and form a pool of ambient sound data to populate our active sound data with

In [41]:
fs = 48000
concatenated_noise_track_fp = "./data/Dataset_concatenated_tracks/Noise/noise_scaled_0.wav"
all_noise_data, _ = librosa.load(concatenated_noise_track_fp, sr=fs , mono=False, dtype=np.float32)

samples_per_frame = int(fs * 0.1)
all_noise_frames = [all_noise_data[: , i:i+samples_per_frame] for i in range(0, len(all_noise_data[0]), samples_per_frame)]
all_noise_frames_array = np.array(all_noise_frames)

print(all_noise_frames_array.shape)
# (how many 100ms frame, 4 channels, sample points for 100ms frame (4800))

(49118, 4, 4800)


### Now we augment each segmeneted audio data file at random with noise data

In [51]:
segmented_data_fp = "./_audio/cleaned_data_0.5s_0.25s/"
active_classes = ['dog', 'impact', 'speech']
fs = 48000
samples_per_frame = int(fs * 0.1)

all_features = []
all_labels = []

for cls in active_classes:
    class_filepath = os.path.join(segmented_data_fp, cls)
    for file in tqdm(os.listdir(class_filepath)):
        if file.endswith('.wav'):
            # Get the clean, segmented audio
            audio_filepath = os.path.join(class_filepath, file)
            audio_data, _ = librosa.load(audio_filepath, sr=fs, mono=False, dtype=np.float32)
            
            # Here, we get the mask to see which 100ms frames to keep/replace
            replacement = get_mask(get_random_numframes_replace())
            
            # Converting the audio data into 100ms frames
            feature_audio_frames = [audio_data[: , i:i+samples_per_frame] for i in range(0, len(audio_data[0]), samples_per_frame)]
            
            augmented_audio = []
            
            # Here we either keep the class audio, or we replace them with a random noise audio
            for idx, keep in enumerate(replacement):
                if keep:
                    augmented_audio.append(feature_audio_frames[idx])
                else:
                    noiseframe_idx = get_random_index()
                    augmented_audio.append(all_noise_frames_array[noiseframe_idx])
                    
            # Combine them all together 
            augmented_audio_array = np.array(augmented_audio)
            augmented_audio_array = np.concatenate(augmented_audio_array, axis = -1)
            
            # Finally, we convert this augmented audio into features
            salsalite_features = extract_features(augmented_audio_array)
            
            # filename --> class_azimuth_index.wav, get the ground truths
            filename_gts = file.replace('.wav' , "")
            gts = filename_gts.split("_")
            
            # This is for active class ground truth
            one_frame_gt = np.zeros(len(active_classes) * 3, dtype=np.float32)
            class_index = active_classes.index(gts[0])
            one_frame_gt[class_index] = 1
            
            # Converting Azimuth into radians and assigning to active class
            gt_azi = int(gts[1])
            # Convert the azimuth from [0, 360) to [-180, 180), taking (0 == 0) and (180 == -180)
            if gt_azi == 330:
                gt_azi = -30
            elif gt_azi == 270:
                gt_azi = -90
            elif gt_azi == 210:
                gt_azi = -150

            azi_rad = gt_azi * np.pi / 180.0 # Convert to radian unit this way
            one_frame_gt[class_index + len(active_classes)] = np.cos(azi_rad) # X-coordinate
            one_frame_gt[class_index + 2 * len(active_classes)] = np.sin(azi_rad) # Y-coordinate
            # Produce the ground truth labels for a single frame, expand the frame dimensions later
            one_frame_gt = np.reshape(one_frame_gt, (1, len(one_frame_gt)))
            
            # This is for noise frame ground truth
            noise_frame_gt = np.zeros(len(active_classes) * 3, dtype=np.float32)
            noise_frame_gt = np.reshape(noise_frame_gt, (1, len(noise_frame_gt)))
            
            # Finally, we combine the active class and noise ground truths 
            gt_result = np.empty((5, len(active_classes)*3))
            for is_kept in replacement:
                if is_kept:
                    gt_result = np.concatenate((gt_result, one_frame_gt), axis=0)
                else:
                    gt_result = np.concatenate((gt_result, noise_frame_gt), axis=0)
            
            all_features.append(salsalite_features)
            all_labels.append(gt_result)

  0%|          | 0/14352 [00:00<?, ?it/s]

 67%|██████▋   | 9602/14352 [13:29<05:55, 13.36it/s]

In [ ]:
final_dataset_storagepath = "./training_datasets/demo_dataset_0.5s_0.25s_wgn20"
feature_fp = os.path.join(final_dataset_storagepath, "augmented_salsalite_features.npy")
np.save(feature_fp, all_features, allow_pickle=True)

label_fp = os.path.join(final_dataset_storagepath, "augmented_salsalite_labels.npy")
np.save(label_fp, all_labels, allow_pickle=True)

In [50]:
all_labels[0][0]

array([2.96439388e-323, 1.14398339e-311, 2.47032823e-323, 3.16202013e-322,
       4.94065646e-324, 1.14392002e-311, 0.00000000e+000, 6.32404027e-322,
       1.14645439e-311])

(49118, 4, 4800)


In [7]:

class_audio_dir = "./_audio/cleaned_data_0.5s_0.25s/dog/"
class_audio_files = os.listdir(class_audio_dir)
num_files = int(len(class_audio_files) * 0.5)
selected_files = random.sample(class_audio_files, num_files)





['dog_90_33.wav', 'dog_90_592.wav', 'dog_210_1331.wav', 'dog_270_1252.wav', 'dog_270_849.wav', 'dog_30_1208.wav', 'dog_30_1920.wav', 'dog_90_1571.wav', 'dog_330_712.wav', 'dog_270_184.wav', 'dog_30_2229.wav', 'dog_150_1429.wav', 'dog_330_327.wav', 'dog_330_1193.wav', 'dog_150_1754.wav', 'dog_30_2013.wav', 'dog_210_1909.wav', 'dog_30_621.wav', 'dog_90_1454.wav', 'dog_330_1951.wav', 'dog_30_2194.wav', 'dog_270_9.wav', 'dog_90_1726.wav', 'dog_150_1122.wav', 'dog_90_1037.wav', 'dog_150_2207.wav', 'dog_150_21.wav', 'dog_210_1266.wav', 'dog_270_1410.wav', 'dog_270_309.wav', 'dog_210_892.wav', 'dog_210_2312.wav', 'dog_270_2232.wav', 'dog_330_1047.wav', 'dog_90_1721.wav', 'dog_150_1003.wav', 'dog_150_1910.wav', 'dog_270_2390.wav', 'dog_150_720.wav', 'dog_150_476.wav', 'dog_210_2310.wav', 'dog_210_621.wav', 'dog_210_796.wav', 'dog_270_145.wav', 'dog_90_1841.wav', 'dog_210_2012.wav', 'dog_90_1587.wav', 'dog_330_278.wav', 'dog_150_1607.wav', 'dog_210_825.wav', 'dog_270_1148.wav', 'dog_150_1600.wa

### Generate data for 